Задача поиска похожих по эмбеддингам
Скачиваем датасет (источник): положительные, отрицательные.

1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

--2022-06-07 17:26:35--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-07 17:26:35--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc89f103c27c13694751c268ee27.dl.dropboxusercontent.com/cd/0/inline/BmwWNWzR1gQcziG43Q3CH9mzRRD5a_yGfK6rlbYuYrRzX-QJ6vsK0pTlCIg-iatBX638qK4Ot9zCzfVwonRHgDKphmD-ZFVzldYLMnS-ktoOyBK5kjHMBu2t2wDTZg-dkWSaZfwna2N0tNTHlJ6O05LIpQSYobMs-X-LqdLKKIgIkA/file# [following]
--2022-06-07 17:26:36--  https://uc89f103c27c13694751c268ee27.dl.dropboxusercontent.com/cd/0/inline/BmwWNWzR1gQcziG43Q3CH9mzRRD5a_yGfK6rlbYuYrRzX-QJ6vsK0pTlCIg-iatBX

In [2]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-07 17:39:57--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2022-06-07 17:39:58--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2041f82317e88971495f6d18e4.dl.dropboxusercontent.com/cd/0/inline/Bmx-9b1KSDT85hpKwQ4m3eKkSPnqp9G0xC75YU8xu3hykumC5nyM_BQS1ZTiPBZ1ysWB1ls-9TUslx7DNtQBIrUtKA1ANdjYT4e2h4jPhr0LFRXs3Ygcw5P-eRnatRL8M1RRsTTmkXQjRWMLdIL27RbExPbiDOc8xL06X7p1A8D9MA/file# [following]
--2022-06-07 17:39:58--  https://uc2041f82317e88971495f6d18e4.dl.dropboxusercontent.com/cd/0/inline/Bmx-9b1KSDT85hpKwQ4m3eKkSPnqp9G0xC75YU8xu3hykumC5nyM_BQS1ZTiPBZ1y

In [1]:
emoticon_dict = {
":)": "счастливый",
":‑)": "счастливый",
":-]": "счастливый",
":-3": "счастливый",
":->": "счастливый",
"8-)": "счастливый",
":-}": "счастливый",
":o)": "счастливый",
":c)": "счастливый",
":^)": "счастливый",
"=]": "счастливый",
"=)": "счастливый",
"<3": "счастливый",
":-(": "грустный",
":(": "грустный",
":c": "грустный",
":<": "грустный",
":[": "грустный",
">:[": "грустный",
":{": "грустный",
">:(": "грустный",
":-c": "грустный",
":-< ": "грустный",
":-[": "грустный",
":-||": "грустный"
}

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import annoy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim.downloader as ap
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm import tqdm_notebook

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/sokolovaeka@ozon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sokolovaeka@ozon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# !pip install annoy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 646 kB 1.4 MB/s eta 0:00:01
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=394717 sha256=35de7b80a002add9cc8f030aca8bdf9d98d628b52202db7ce9c100739278be63
  Stored in directory: /home/sokolovaeka@ozon/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [3]:
positive = pd.read_csv('positive.csv', delimiter=';', usecols=[3,4], names=['tweet', 'intent'])
negative = pd.read_csv('negative.csv', delimiter=';', usecols=[3,4], names=['tweet',  'intent'])

df = pd.concat([positive, negative])

In [5]:
negative.intent.value_counts()

-1    111923
Name: intent, dtype: int64

In [6]:
import nltk
nltk.download('stopwords')

def user_del2(tweet):
    """Delete @user from tweet"""       
    return re.sub(r'@[\w]*', '', tweet)


def change_from_dict(tweet, dictionary): 
    """Chenge element for value from dict if exists"""
    full_text = [dictionary.get(word) or word for word in tweet.split(' ')]
    return ' '.join(full_text)

def tweets_preprocessing(example): 

    # Удалим @user из всех твитов
    example = user_del2(example)
    # Изменим регистр твитов на нижний 
    example = example.lower()
    # Заменим сокращения с апострофами 
#     example = change_from_dict(example, short_word_dict)
    # Заменим эмотиконы
    example = change_from_dict(example, emoticon_dict)
    # Заменим пунктуацию на пробелы
    example = re.sub(r'[^\w\s]', ' ', example)
    # Заменим спец. символы на пробелы
    example = re.sub(r'[^a-zA-Zа-яёA-ЯЁ0-9]', ' ', example)
    # Заменим числа на пробелы
    example = re.sub(r'[^a-zA-Zа-яёA-ЯЁ]', ' ', example)
    # Удалим из текста слова длиной в 2 символа
    example = [w for w in example.split() if len(w)>2]
    # Удалим стоп-слова
    stop_words = get_stop_words("ru") + stopwords.words('russian')
    example = [w for w in example if w not in stop_words] 
    # Приведем к нормальной форме
    morph = MorphAnalyzer()
    example = [morph.parse(w)[0].normal_form for w in example]  
    
    return example

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sokolovaeka@ozon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df['preprocessed'] = df.tweet.apply(tweets_preprocessing)
df.reset_index(drop=True)

df.head()

,tweet,intent,preprocessed
0,"@first_timee хоть я и школота, но поверь, у на...",1,"[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...",1,"[таки, похожий, мальчик, равно]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,"[идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"[угол, сидеть, погибать, голод, порция, взять,..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"[страшилка, блин, посмотреть, часть, создаться..."


In [10]:
# df.to_csv('prepared_tweets.csv')
# df = pd.read_csv('prepared_tweets.csv')
df = df[['tweet', 'intent', 'preprocessed']]
df.head(2)

,tweet,intent,preprocessed
0,"@first_timee хоть я и школота, но поверь, у на...",1,"['школотый', 'поверь', 'самый', 'общество', 'п..."
1,"Да, все-таки он немного похож на него. Но мой ...",1,"['таки', 'похожий', 'мальчик', 'равно']"


In [11]:
tweets = [tweet for tweet in df['preprocessed'].values if len(tweet)>2]
tweets[:3]

["['школотый', 'поверь', 'самый', 'общество', 'профилировать', 'предмет', 'тип']",
 "['таки', 'похожий', 'мальчик', 'равно']",
 "['идиотка', 'испугаться']"]

In [12]:
modelW2V = Word2Vec(sentences=tweets, vector_size=300, window=5, min_count=1)
modelFT = FastText(sentences=tweets, vector_size=300, min_count=1, window=5, workers=8)

In [13]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in df['preprocessed']:
    n_w2v = 0
    n_ft = 0
#     spls = line.split("\t")
    index_map[counter] = line

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in line:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)


True

In [14]:
def get_response(tweet, index, model, index_map):
    tweet = tweets_preprocessing(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [df.tweet.values[i] for i in answers]

In [27]:
test_tweet = 'Сегодня хорошая погода'

In [28]:
get_response(test_tweet, w2v_index, modelW2V, index_map)

['Только сейчас с трени пришла......на улице наступила настоящая зима.Красотень то какая:*',
 'Зашла в аудиторию,как получила под кофту снежка!от Оли Пупач) спасибо)',
 '@Radkat100 @OlyaValsbaston измени установку: должно быть отлично)))',
 '@Kisa_Kris если через контору снимать не меньше 12 это точно , 2 недели назад узнавали ) а если сами , то там уже на совесть цены',
 'скиди, ты не тупая, а это я СТО ПРОЦ, если еще будет неправильно то, что она у меня списала........ то мне не жить, хотя это даже КУЛЛ)']

In [29]:
get_response(test_tweet, ft_index, modelFT, index_map)

['@Valery_Valeriya пахахахахахахахахахахахахахахахахахахах :DDDDDDDD',
 '@kaa_musakaeva хахаахаххахахаххххахахаахахахахх))))))',
 '@v_strepetov @vlad__nvp  АХАХАХАХАХХАХАХАХАХАХАХАХАХАХАХА !):DD',
 '@HappyNataly_Sib ахахахахахахахахахахахах лишь бы не меньше)))',
 'RT @MAPCOKEMOB_MHE: @yulechka_minion аххахаххахахахах \n:D']

Модель word2vec показала более-менее похожие по смыслу твиты, FastText что-то плохо отработала